In [1]:
# Preprocessing Data for suggroate model 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import seaborn as sns
import sklearn
import imblearn
import matplotlib.pyplot as plt
import time
import sklearn.metrics as m
import xgboost as xgb
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# read the data
data_target = pd.read_csv('../datasets/target_model/CICIDS2017/df_target_0311.csv')
data_sugrrogate = pd.read_csv('../datasets/surrogate_model/CICIDS2017/df_surrogate_0311.csv') # datasets\surrogate_model\CICIDS2017\df_surrogate_0311.csv
df = pd.concat([data_target, data_sugrrogate], axis=0)
df



,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Binary_class,attack_type
0,80,5435785,4,0,24,0,6,6,6.000000,0.000000,...,0.0,952,952,5434833.0,0.0,5434833,5434833,DDoS,malicious,DOS
1,6668,26,1,1,2,6,2,2,2.000000,0.000000,...,0.0,0,0,0.0,0.0,0,0,PortScan,malicious,PortScan
2,53,23475,1,1,70,152,70,70,70.000000,0.000000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
3,80,37553,2,0,0,0,0,0,0.000000,0.000000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
4,53,71220,4,4,172,384,43,43,43.000000,0.000000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1261180,53,23712,2,2,82,190,41,41,41.000000,0.000000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
1261181,80,97890396,6,6,357,11595,357,0,59.500000,145.744640,...,0.0,292,292,97900000.0,0.0,97900000,97900000,DoS Hulk,NaN,DoS
1261182,60040,32,1,1,0,0,0,0,0.000000,0.000000,...,0.0,0,0,0.0,0.0,0,0,BENIGN,benign,benign
1261183,443,11408070,19,21,10829,22666,3638,0,569.947368,1303.224781,...,0.0,1408037,1408037,9999813.0,0.0,9999813,9999813,BENIGN,benign,benign


In [3]:
# Preprocessing Data
# Removing whitespaces in column names.
col_names = [col.replace(' ', '') for col in df.columns]
df.columns = col_names


# Replace weird characters
label_names = df['Label'].unique()

import re
label_names = [re.sub("[^a-zA-Z ]+", "", l) for l in label_names]
label_names = [re.sub("[\s\s]", '_', l) for l in label_names]
label_names = [lab.replace("__", "_") for lab in label_names]
label_names, len(label_names)

# Replacing 'Label' column values with new readable values.
labels = df['Label'].unique()
for i in range(0,len(label_names)):
    df['Label'] = df['Label'].replace({labels[i] : label_names[i]})
    
df['Label'].unique()

df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

In [4]:
df.value_counts('Label')

Label
BENIGN                      2095057
DDoS                         128014
PortScan                      90694
DoS_GoldenEye                 10286
FTPPatator                     5931
DoS_slowloris                  5385
DoS_Slowhttptest               5228
SSHPatator                     3219
Bot                            1948
Web_Attack_Brute_Force         1470
Web_Attack_XSS                  652
Infiltration                     36
Web_Attack_Sql_Injection         21
Heartbleed                       11
Name: count, dtype: int64

In [5]:
df_benign = df[df['Label'] == 'BENIGN']
df_benign = df_benign.sample(frac=0.02).reset_index(drop=True)
df_DOS_GoldenEye = df[df['Label'] == 'DoS_GoldenEye']
df_DOS_slowloris = df[df['Label'] == 'DoS_slowloris']
df_DOS_slowhttptest = df[df['Label'] == 'DoS_slowhttptest']
# df_DOS_GoldenEye

In [6]:
df = pd.concat([df_benign, df_DOS_GoldenEye, df_DOS_slowloris, df_DOS_slowhttptest], axis=0)
# df = pd.concat([df_benign, df_DOS_slowloris, df_DOS_slowhttptest], axis=0)
df = df.rename(columns={'Binary_class': 'binary_classification'})
df

,DestinationPort,FlowDuration,TotalFwdPackets,TotalBackwardPackets,TotalLengthofFwdPackets,TotalLengthofBwdPackets,FwdPacketLengthMax,FwdPacketLengthMin,FwdPacketLengthMean,FwdPacketLengthStd,...,ActiveStd,ActiveMax,ActiveMin,IdleMean,IdleStd,IdleMax,IdleMin,Label,binary_classification,attack_type
0,443,119970779,6,8,123,105,35,6,20.500000,15.883954,...,2.133558e+02,32779,32341,23961591.6,1.920725e+07,45003096,9932269,BENIGN,benign,benign
1,80,117591768,99,122,2547,205796,408,0,25.727273,94.293811,...,7.352316e+05,2453564,14977,10000000.0,1.863627e+03,10000000,10000000,BENIGN,benign,benign
2,62379,609363,2,2,37,12,37,0,18.500000,26.162951,...,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN,benign,benign
3,123,64039358,2,2,96,96,48,48,48.000000,0.000000,...,0.000000e+00,36695,36695,63965838.0,0.000000e+00,63965838,63965838,BENIGN,benign,benign
4,123,57329,1,1,48,48,48,48,48.000000,0.000000,...,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN,benign,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255279,80,102645877,14,3,2541,6,231,0,181.500000,98.363337,...,4.529980e+06,6406845,486,19200000.0,1.880000e+07,51300000,5966340,DoS_slowloris,malicious,DoS
1258044,80,3003648,3,0,0,0,0,0,0.000000,0.000000,...,0.000000e+00,0,0,0.0,0.000000e+00,0,0,DoS_slowloris,malicious,DoS
1258419,80,3003925,3,0,0,0,0,0,0.000000,0.000000,...,0.000000e+00,0,0,0.0,0.000000e+00,0,0,DoS_slowloris,malicious,DoS
1260583,80,102690607,14,3,2541,6,231,0,181.500000,98.363337,...,4.532999e+06,6411372,744,19300000.0,1.880000e+07,51300000,5743827,DoS_slowloris,malicious,DoS


In [7]:
df.columns

Index(['DestinationPort', 'FlowDuration', 'TotalFwdPackets',
       'TotalBackwardPackets', 'TotalLengthofFwdPackets',
       'TotalLengthofBwdPackets', 'FwdPacketLengthMax', 'FwdPacketLengthMin',
       'FwdPacketLengthMean', 'FwdPacketLengthStd', 'BwdPacketLengthMax',
       'BwdPacketLengthMin', 'BwdPacketLengthMean', 'BwdPacketLengthStd',
       'FlowBytes/s', 'FlowPackets/s', 'FlowIATMean', 'FlowIATStd',
       'FlowIATMax', 'FlowIATMin', 'FwdIATTotal', 'FwdIATMean', 'FwdIATStd',
       'FwdIATMax', 'FwdIATMin', 'BwdIATTotal', 'BwdIATMean', 'BwdIATStd',
       'BwdIATMax', 'BwdIATMin', 'FwdPSHFlags', 'BwdPSHFlags', 'FwdURGFlags',
       'BwdURGFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s',
       'BwdPackets/s', 'MinPacketLength', 'MaxPacketLength',
       'PacketLengthMean', 'PacketLengthStd', 'PacketLengthVariance',
       'FINFlagCount', 'SYNFlagCount', 'RSTFlagCount', 'PSHFlagCount',
       'ACKFlagCount', 'URGFlagCount', 'CWEFlagCount', 'ECEFlagCount',
       '

In [8]:
# drop columns
df_feature = df[['FlowDuration', 'TotalFwdPackets', 'TotalBackwardPackets', 'TotalLengthofFwdPackets', 'TotalBackwardPackets', 
                 'FwdPacketLengthMin', 'FwdPacketLengthStd', 'BwdPacketLengthMax', 'BwdPacketLengthMean'
                 ,'BwdPacketLengthStd', 'FlowBytes/s', 'FlowPackets/s', 'FlowIATStd', 'FwdIATTotal', 'FwdIATMean', 'FwdIATMax', 'BwdIATMean', 'BwdIATStd'
                 , 'BwdIATMax', 'BwdIATMin', 'BwdPSHFlags', 'FwdHeaderLength', 'BwdHeaderLength', 'FwdPackets/s', 'BwdPackets/s', 'MaxPacketLength', 'PacketLengthStd'
                 , 'FINFlagCount', 'SYNFlagCount', 'ACKFlagCount', 'Down/UpRatio', 'AvgBwdSegmentSize', 'FwdHeaderLength.1', 'SubflowFwdPackets', 
                 'SubflowFwdBytes', 'IdleStd', 
                 'SubflowBwdPackets', 'SubflowBwdBytes', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'ActiveStd', 'ActiveMax', 'Label', 'binary_classification']]

In [9]:
df_feature.columns = ['FlowDuration', 'TotFwdPkts', 'TotBwdPkts', 'TotLenFwdPkts', 'TotLenBwdPkts', 'FwdPktLenMin', 'FwdPktLenStd', 'BwdPktLenMax', 'BwdPktLenMean'
                 ,'BwdPktLenStd', 'FlowByts/s', 'FlowPkts/s', 'FlowIATStd', 'FwdIATTot', 'FwdIATMean', 'FwdIATMax', 'BwdIATMean', 'BwdIATStd'
                 , 'BwdIATMax', 'BwdIATMin', 'BwdPSHFlags', 'FwdHeaderLen', 'BwdHeaderLen', 'FwdPkts/s', 'BwdPkts/s', 'PktLenMax', 'PktLenStd'
                 , 'FINFlagCnt', 'SYNFlagCnt', 'ACKFlagCnt', 'Down/UpRatio', 'BwdSegSizeAvg', 'FwdHeaderLen', 'SubflowFwdPkts', 'SubflowFwdByts', 'IdleStd', 
                 'SubflowBwdPkts', 'SubflowBwdByts', 'InitBwdWinByts', 'FwdActDataPkts', 'ActiveStd', 'ActiveMax', 'Label', 'binary_classification']

In [10]:
df_feature

,FlowDuration,TotFwdPkts,TotBwdPkts,TotLenFwdPkts,TotLenBwdPkts,FwdPktLenMin,FwdPktLenStd,BwdPktLenMax,BwdPktLenMean,BwdPktLenStd,...,SubflowFwdByts,IdleStd,SubflowBwdPkts,SubflowBwdByts,InitBwdWinByts,FwdActDataPkts,ActiveStd,ActiveMax,Label,binary_classification
0,119970779,6,8,123,8,6,15.883954,31,13.125000,15.008926,...,123,1.920725e+07,8,105,55480,5,2.133558e+02,32779,BENIGN,benign
1,117591768,99,122,2547,122,0,94.293811,4344,1686.852459,1037.995945,...,2547,1.863627e+03,122,205796,1140,7,7.352316e+05,2453564,BENIGN,benign
2,609363,2,2,37,2,0,26.162951,6,6.000000,0.000000,...,37,0.000000e+00,2,12,0,1,0.000000e+00,0,BENIGN,benign
3,64039358,2,2,96,2,48,0.000000,48,48.000000,0.000000,...,96,0.000000e+00,2,96,-1,1,0.000000e+00,36695,BENIGN,benign
4,57329,1,1,48,1,48,0.000000,48,48.000000,0.000000,...,48,0.000000e+00,1,48,-1,0,0.000000e+00,0,BENIGN,benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255279,102645877,14,3,2541,3,0,98.363337,6,2.000000,3.464102,...,2541,1.880000e+07,3,6,0,11,4.529980e+06,6406845,DoS_slowloris,malicious
1258044,3003648,3,0,0,0,0,0.000000,0,0.000000,0.000000,...,0,0.000000e+00,0,0,-1,0,0.000000e+00,0,DoS_slowloris,malicious
1258419,3003925,3,0,0,0,0,0.000000,0,0.000000,0.000000,...,0,0.000000e+00,0,0,-1,0,0.000000e+00,0,DoS_slowloris,malicious
1260583,102690607,14,3,2541,3,0,98.363337,6,2.000000,3.464102,...,2541,1.880000e+07,3,6,0,11,4.532999e+06,6411372,DoS_slowloris,malicious


In [11]:
# save the data
df_feature.to_csv('../datasets/target_model/CICIDS2017/target_model_CICIDS2017data.csv', index=False) #datasets\target_model\CICIDS2018